In [ ]:
cd("../TestDrive")

In [ ]:
] activate . 

In [ ]:
include("src/TestDrive.jl")

In [ ]:
# User-specified fake migration matrices 
migrate1 = rand((6*(4+8+6+1+6)), 2, 2) 
migrate2 = rand((6*(4+8+6+1+6)), 2, 2)

# User-named nodes  
firstnode = :FirstNode
secondnode = :SecondNode

# User-constructed nodes: [name, gene_data, stages, location, migration_matrix]
example_nodes = [Node(:FirstNode, genetics_mcr, stages_aedes, temp_none, (37.87, 122.27), migrate1),
    Node(:SecondNode, genetics_mcr, stages_aedes, temp_none, (35.87, 120.27), migrate2)]

# User-constructed network: [nodes, total states (# genotypes x stages in each lifestage)] 
example_network = Network(example_nodes, 150)
;

In [ ]:
# Create init network function 
function init_network!(network::Network)
    genetics = get_genetics(example_network.all_nodes[1])
    n_genes = length(genetics.all_genotypes)
    
    n_nodes = length(network.all_nodes)
    
    # Way to improve? n_genes = accounts for female subtypes; 
    # But sampling first node not necessarily always accurate since could differ. 
    n_states= sum(get_substages(network.all_nodes[1])) + n_genes
    
    u0 = Array{Float64,3}(undef, n_states, n_genes, n_nodes)
    
    for (index,node) in enumerate(network.all_nodes)
        
        # How to avoid hardcoding + make it such that different nodes can 
        # start with different genetic types (beyond individual init_node runs)
        gene_index = 4
        
        u0[:,:,index] = init_node!(node, gene_index)
        
    end
    
    return u0

end

# Initialize all nodes in network simultaneously 
u0 = init_network!(example_network)
;


In [ ]:
tspan = (0.0,10.0) 
controls = [0.0] # necessary even when not using controls bc of the way function is written 
ode_model = diffeq.ODEProblem(population_model_network, u0, tspan, (example_network, controls))

In [ ]:
ode_model.problem_type

In [ ]:
alg = OrdinaryDiffEq.Tsit5() 
sol_ode = diffeq.solve(ode_model, alg)
;

In [ ]:
#sol_ode.u[end][:,4]
#sol_ode.u[end]
sol_ode.u[end][:,:,2]
;

In [ ]:
using Plots
gr()
;

In [ ]:
plot(sol_ode, title = "Migration: Two-Node MCR Population at Equilibrium") 

In [ ]:
tspan = (0.0,300.0) 
controls = [0.0]

# Construct ODE control problem 
control_model = diffeq.ODEProblem(population_model_network, u0, tspan, (example_network, controls))

In [ ]:
#alg = Vern8()
#alg = OrdinaryDiffEq.Tsit5() 
alg = Sundials.CVODE_Adams()
#alg = CVODE_BDF()

sol_control = diffeq.solve(control_model, alg, callback = cbs, tstops = tstop)
;

In [ ]:
sol_control.u[end]
;

In [ ]:
# Question - full plotting routines here in Example file or elsewhere? Check PVCR example. 
# Question - use other plotting packages to make it nice; which? and choose appropriate backend. 

plot(sol_control, title = "Migration: Two-Node MCR Population with Releases")